In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [2]:
data1 = pd.read_csv("project3_dataset1.txt",sep="\t", header=None)
data2 = pd.read_csv("project3_dataset2.txt",sep="\t", header=None)

One hot encode dataset 2 as there are some labeled data

In [3]:
def one_hot(row):
    return int(row[4]=='Present')
data2[4] = data2.apply(lambda row:one_hot(row), axis=1)
data2

0     1      2      3  4   5      6      7   8  9
0    132  6.20   6.47  36.21  1  62  30.77  14.14  45  0
1    123  0.05   4.61  13.69  0  51  23.23   2.78  16  0
2    128  0.50   3.70  12.81  1  66  21.25  22.73  28  0
3    114  9.60   2.51  29.18  0  49  25.67  40.63  46  0
4    150  0.30   6.38  33.99  1  62  24.64   0.00  50  0
..   ...   ...    ...    ... ..  ..    ...    ...  .. ..
457  114  4.08   4.59  14.60  1  62  23.11   6.72  58  1
458  136  2.80   2.53   9.28  1  61  20.70   4.55  25  0
459  138  4.50   2.85  30.11  0  55  24.78  24.89  56  1
460  170  7.60   5.50  37.83  1  42  37.41   6.17  54  1
461  128  0.00  10.58  31.81  1  46  28.41  14.66  48  0

[462 rows x 10 columns]

In [4]:
print(f"data1 shape = {data1.shape}")
print(f"data2 shape = {data2.shape}")

data1 shape = (569, 31)
data2 shape = (462, 10)


-> Accuracy, Precision, Recall, F-1 measure, and AUC (area under the curve)

In [5]:
from sklearn.preprocessing import MinMaxScaler
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score


def run_log_regr(X, X_test_orig, y, y_test_orig, penalty_type='l2', C_val=1.0):
    
    kf = StratifiedKFold(n_splits=10, random_state=0, shuffle=True)

    ct = 0

    fold_results = []

    # Do 10 fold cross validation.

    for train_index, val_index in kf.split(X, y):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        clf = LogisticRegression(penalty=penalty_type, C=C_val, random_state=0, max_iter=10000)
        clf.fit(X_train, y_train)

        pred_val = clf.predict(X_val)

        accr = accuracy_score(y_val, pred_val)
        precision = precision_score(y_val, pred_val)
        recall = recall_score(y_val, pred_val)
        f1 = f1_score(y_val, pred_val)
        roc = roc_auc_score(y_val, pred_val, average='weighted')

        print(f"Fold: {ct}. Accuracy = {accr:.3f}, precision = {precision:.3f}, recall = {recall:.3f}, f1 = {f1:.3f}, AUC = {roc:.3f}")

        fold_results.append([accr, precision, recall, f1, roc])

        ct+=1
    fold_results = np.array(fold_results)
    fold_results = np.sum(fold_results, axis=0)/ct

    print(f"Average across folds: \nAccuracy = {fold_results[0]:.3f}, precision = {fold_results[1]:.3f}, recall = {fold_results[2]:.3f}, f1 = {fold_results[3]:.3f}, AUC = {fold_results[4]:.3f}")

    pred_test_orig = clf.predict(X_test_orig)

    accr_test = accuracy_score(y_test_orig, pred_test_orig)
    precision_test = precision_score(y_test_orig, pred_test_orig)
    recall_test = recall_score(y_test_orig, pred_test_orig)
    f1_test = f1_score(y_test_orig, pred_test_orig)
    roc_test = roc_auc_score(y_test_orig, pred_test_orig, average='weighted')
    results_test = [accr_test, precision_test, recall_test, f1_test, roc_test]
    print(f"Test set results: \nAccuracy = {results_test[0]:.3f}, precision = {results_test[1]:.3f}, recall = {results_test[2]:.3f}, f1 = {results_test[3]:.3f}, AUC = {results_test[4]:.3f}")



def get_data(d1):
    # Preprocess all the data.
    # This involves normalizing the data using MinMaxScaler
    # and splitting the data into train and test sets.

    X = d1[[i for i in range(d1.shape[1]-1)]].to_numpy()
    y = d1[[ d1.shape[1]-1 ]].to_numpy()
    
    c = Counter(y.ravel())
    print(f"Distribution: {c}")
    
    y = y.ravel()
    
    mms = MinMaxScaler()
    X = mms.fit_transform(X)
    X = np.hstack(( np.ones([len(X), 1]), X))

    print(f"X.shape, y.shape = {X.shape}, {y.shape}")
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

In [6]:

print(f"\n============ Dataset 1 ============\n")
X_train, X_test, y_train, y_test = get_data(data1)
met = run_log_regr(X_train, X_test, y_train, y_test)

print("="*50)

print(f"\n============ Dataset 2 ============\n")

X_train, X_test, y_train, y_test = get_data(data2)
met = run_log_regr(X_train, X_test, y_train, y_test)



============ Dataset 1 ============

Distribution: Counter({0: 357, 1: 212})
X.shape, y.shape = (569, 31), (569,)
Fold: 0. Accuracy = 0.978, precision = 1.000, recall = 0.941, f1 = 0.970, AUC = 0.971
Fold: 1. Accuracy = 0.978, precision = 1.000, recall = 0.941, f1 = 0.970, AUC = 0.971
Fold: 2. Accuracy = 0.913, precision = 1.000, recall = 0.765, f1 = 0.867, AUC = 0.882
Fold: 3. Accuracy = 0.978, precision = 1.000, recall = 0.941, f1 = 0.970, AUC = 0.971
Fold: 4. Accuracy = 0.978, precision = 1.000, recall = 0.941, f1 = 0.970, AUC = 0.971
Fold: 5. Accuracy = 0.978, precision = 1.000, recall = 0.941, f1 = 0.970, AUC = 0.971
Fold: 6. Accuracy = 0.956, precision = 1.000, recall = 0.875, f1 = 0.933, AUC = 0.938
Fold: 7. Accuracy = 0.933, precision = 1.000, recall = 0.812, f1 = 0.897, AUC = 0.906
Fold: 8. Accuracy = 0.956, precision = 1.000, recall = 0.875, f1 = 0.933, AUC = 0.938
Fold: 9. Accuracy = 1.000, precision = 1.000, recall = 1.000, f1 = 1.000, AUC = 1.000
Average across folds: 
Ac